In [ ]:
!wget https://personal.utdallas.edu/~pxn210006/cnn_dailymail/CNN_Dailymail_Data.zip
!unzip /content/CNN_Dailymail_Data.zip -d /content/json_data

--2022-04-27 22:45:08--  https://personal.utdallas.edu/~pxn210006/cnn_dailymail/CNN_Dailymail_Data.zip
Resolving personal.utdallas.edu (personal.utdallas.edu)... 129.110.46.112
Connecting to personal.utdallas.edu (personal.utdallas.edu)|129.110.46.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40060395 (38M) [application/zip]
Saving to: ‘CNN_Dailymail_Data.zip’

CNN_Dailymail_Data. 100%[===================>]  38.20M   832KB/s    in 49s     

2022-04-27 22:45:58 (804 KB/s) - ‘CNN_Dailymail_Data.zip’ saved [40060395/40060395]

Archive:  /content/CNN_Dailymail_Data.zip
  inflating: /content/json_data/CNN_Dailymail_Data.json  


In [ ]:
import json

with open('/content/json_data/CNN_Dailymail_Data.json') as dataFile:

    for cnnJsonData in dataFile:
        processedData = json.loads(cnnJsonData)

        vocabulary_index_mapping = processedData["vocabulary_index_mapping"]
        embd = processedData["embeddings_word"]
        train_article_batches = processedData["train_article_batches"]
        test_article_batches = processedData["test_article_batches"]
        validation_article_batches = processedData["validation_article_batches"]
        train_summary_batches = processedData["train_summary_batches"]
        test_summary_batches = processedData["test_summary_batches"]
        validation_summary_batches= processedData["validation_summary_batches"]
        train_article_batches_article_length = processedData["train_article_batches_article_length"]
        validation_article_batches_article_length = processedData["validation_article_batches_article_length"]
        test_article_batches_article_length = processedData["test_article_batches_article_length"]
        train_summary_batches_summary_length = processedData["train_summary_batches_summary_length"]
        validation_summary_batches_summary_length = processedData["validation_summary_batches_summary_length"]
        test_summary_batches_summary_length = processedData["test_summary_batches_summary_length"]

        break
        
index_vocabulary_mapping = {vocabValue:vocabKey for vocabKey,vocabValue in vocabulary_index_mapping.items()}

In [ ]:
# All hyper parameters 
hid_lay_cnn = 300
learnRate_cnn = 0.001
epochs = 1
MaxSummaryLen_CNN = 31 
Attention_Dimension_Size = 5 
winDowCNNSize = 2*Attention_Dimension_Size+1
regL2Param=1e-6

In [ ]:
#loading tensorflow
import tensorflow.compat.v1 as version1_compat_tensorflow 
version1_compat_tensorflow.reset_default_graph()
version1_compat_tensorflow.disable_v2_behavior()
version1_compat_tensorflow.disable_eager_execution()

embedding_dimensions = len(embd[0])

article_tensor_cnn = version1_compat_tensorflow.placeholder(version1_compat_tensorflow.int32, [None, None])
embed_tensor_cnn = version1_compat_tensorflow.placeholder(version1_compat_tensorflow.float32, [len(vocabulary_index_mapping),embedding_dimensions])
summary_true_tensor_len_cnn = version1_compat_tensorflow.placeholder(version1_compat_tensorflow.int32, [None])
sumCNN_Tensor = version1_compat_tensorflow.placeholder(version1_compat_tensorflow.int32,[None, None])
train_tensor_cnn = version1_compat_tensorflow.placeholder(version1_compat_tensorflow.bool)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#defining dropout function 
def CnnDropout(x,rate,training):
    return version1_compat_tensorflow.cond(train_tensor_cnn,
                    lambda: version1_compat_tensorflow.nn.dropout(x,rate=0.3),
                    lambda: x)

In [ ]:
# creating embedding lookup and getting dropout
CnnEmbeddedSummary = version1_compat_tensorflow.nn.embedding_lookup(embed_tensor_cnn, article_tensor_cnn)

CnnEmbeddedSummary = CnnDropout(CnnEmbeddedSummary,rate=0.3,training=train_tensor_cnn)

In [ ]:
# defining the lstm model architecture with all hyper parameters
def CnnLstm(x,CnnHiddenState,CLNcell,CnninputDim,hid_lay_cnn,scope):
    
    with version1_compat_tensorflow.variable_scope(scope,reuse=version1_compat_tensorflow.AUTO_REUSE):
        
        CnnLstmWeight = version1_compat_tensorflow.get_variable("w", shape=[4,CnninputDim,hid_lay_cnn],
                                    dtype=version1_compat_tensorflow.float32,
                                    trainable=True,
                                    initializer=version1_compat_tensorflow.glorot_uniform_initializer())
        
        CnnLstmU = version1_compat_tensorflow.get_variable("u", shape=[4,hid_lay_cnn,hid_lay_cnn],
                            dtype=version1_compat_tensorflow.float32,
                            trainable=True,
                            initializer=version1_compat_tensorflow.glorot_uniform_initializer())
        
        CnnLstmB = version1_compat_tensorflow.get_variable("bias", shape=[4,1,hid_lay_cnn],
                    dtype=version1_compat_tensorflow.float32,
                    trainable=True,
                    initializer=version1_compat_tensorflow.zeros_initializer())
        
    CnnGateInput= version1_compat_tensorflow.nn.sigmoid( version1_compat_tensorflow.matmul(x,CnnLstmWeight[0]) + version1_compat_tensorflow.matmul(CnnHiddenState,CnnLstmU[0]) + CnnLstmB[0])
    CnnGateForget = version1_compat_tensorflow.nn.sigmoid( version1_compat_tensorflow.matmul(x,CnnLstmWeight[1]) + version1_compat_tensorflow.matmul(CnnHiddenState,CnnLstmU[1]) + CnnLstmB[1])
    CnnGateOutput = version1_compat_tensorflow.nn.sigmoid( version1_compat_tensorflow.matmul(x,CnnLstmWeight[2]) + version1_compat_tensorflow.matmul(CnnHiddenState,CnnLstmU[2]) + CnnLstmB[2])
    CnnCell_ = version1_compat_tensorflow.nn.tanh( version1_compat_tensorflow.matmul(x,CnnLstmWeight[3]) + version1_compat_tensorflow.matmul(CnnHiddenState,CnnLstmU[3]) + CnnLstmB[3])
    CnnCell = CnnGateForget*CLNcell + CnnGateInput*CnnCell_
    CnnHiddenState = CnnGateOutput*version1_compat_tensorflow.tanh(CnnCell)
    
    return CnnHiddenState, CnnCell

In [ ]:
# defining architecture for forward encoding
seq_length = version1_compat_tensorflow.shape(CnnEmbeddedSummary)[1]
batch_size = version1_compat_tensorflow.shape(CnnEmbeddedSummary)[0] 

i=0
CNNHidden=version1_compat_tensorflow.zeros([batch_size, hid_lay_cnn], dtype=version1_compat_tensorflow.float32)
CLNcell=version1_compat_tensorflow.zeros([batch_size, hid_lay_cnn], dtype=version1_compat_tensorflow.float32)
CnnHiddenForward=version1_compat_tensorflow.TensorArray(size=seq_length, dtype=version1_compat_tensorflow.float32)

CnnEmbeddedSummary_t = version1_compat_tensorflow.transpose(CnnEmbeddedSummary,[1,0,2]) 


def conditonsForward(i, CNNHidden, CLNcell, CnnHiddenForward):
    return i < seq_length

def bodyForward(i, CNNHidden, CLNcell, CnnHiddenForward):
    x = CnnEmbeddedSummary_t[i]
    
    CNNHidden,CLNcell = CnnLstm(x,CNNHidden,CLNcell,embedding_dimensions,hid_lay_cnn,scope="forward_encoder")
    CnnHiddenForward = CnnHiddenForward.write(i, CNNHidden)

    return i+1, CNNHidden, CLNcell, CnnHiddenForward

_, _, _, CnnHiddenForward = version1_compat_tensorflow.while_loop(conditonsForward, bodyForward, [i, CNNHidden, CLNcell, CnnHiddenForward])

In [ ]:
# defining architecture for backward encoding
i=seq_length-1
CnnHidden=version1_compat_tensorflow.zeros([batch_size, hid_lay_cnn], dtype=version1_compat_tensorflow.float32)
CLNcell=version1_compat_tensorflow.zeros([batch_size, hid_lay_cnn], dtype=version1_compat_tensorflow.float32)
CnnHiddenBackward=version1_compat_tensorflow.TensorArray(size=seq_length, dtype=version1_compat_tensorflow.float32)

def conditonsBackward(i, CnnHidden, CLNcell, CnnHiddenBackward):
    return i >= 0

def bodyBackward(i, CnnHidden, CLNcell, CnnHiddenBackward):
    x = CnnEmbeddedSummary_t[i]
    CnnHidden,cell = CnnLstm(x,CnnHidden,CLNcell,embedding_dimensions,hid_lay_cnn,scope="backward_encoder")
    CnnHiddenBackward = CnnHiddenBackward.write(i,CnnHidden)

    return i-1, CnnHidden, CLNcell, CnnHiddenBackward

_, _, _, CnnHiddenBackward = version1_compat_tensorflow.while_loop(conditonsBackward, bodyBackward, [i, CnnHidden, CLNcell, CnnHiddenBackward])

In [ ]:
# merging forward and backward hidden states
CnnHiddenForward = CnnHiddenForward.stack()
CnnHiddenBackward = CnnHiddenBackward.stack()

CnnEncoderStates = version1_compat_tensorflow.concat([CnnHiddenForward,CnnHiddenBackward],axis=-1)
CnnEncoderStates = version1_compat_tensorflow.transpose(CnnEncoderStates,[1,0,2])

CnnEncoderStates = CnnDropout(CnnEncoderStates,rate=0.3,training=train_tensor_cnn)

FinalEncodedState = CnnDropout(version1_compat_tensorflow.concat([CnnHiddenForward[-1],CnnHiddenBackward[-1]],axis=-1),rate=0.3,training=train_tensor_cnn)

In [ ]:
# defining architecture for attention scoring
def CnnAttentionScore(CnnEncoderStates,CnnDecoderHiddenState,scope="attention_score"):
    
    with version1_compat_tensorflow.variable_scope(scope,reuse=version1_compat_tensorflow.AUTO_REUSE):
        WeightA = version1_compat_tensorflow.get_variable("Wa", shape=[2*hid_lay_cnn,2*hid_lay_cnn],
                                    dtype=version1_compat_tensorflow.float32,
                                    trainable=True,
                                    initializer=version1_compat_tensorflow.glorot_uniform_initializer())
        
    CnnEncoderStates = version1_compat_tensorflow.reshape(CnnEncoderStates,[batch_size*seq_length,2*hid_lay_cnn])
    
    CnnEncoderStates = version1_compat_tensorflow.reshape(version1_compat_tensorflow.matmul(CnnEncoderStates,WeightA),[batch_size,seq_length,2*hid_lay_cnn])
    CnnDecoderHiddenState = version1_compat_tensorflow.reshape(CnnDecoderHiddenState,[batch_size,2*hid_lay_cnn,1])
    
    return version1_compat_tensorflow.reshape(version1_compat_tensorflow.matmul(CnnEncoderStates,CnnDecoderHiddenState),[batch_size,seq_length])

In [ ]:
# defining function for local attention
def align(CnnEncoderStates, CnnDecoderHiddenState,scope="attention"):
    
    with version1_compat_tensorflow.variable_scope(scope,reuse=version1_compat_tensorflow.AUTO_REUSE):
        W_par = version1_compat_tensorflow.get_variable("Wp", shape=[2*hid_lay_cnn,128],
                                    dtype=version1_compat_tensorflow.float32,
                                    trainable=True,
                                    initializer=version1_compat_tensorflow.glorot_uniform_initializer())
        
        V_par = version1_compat_tensorflow.get_variable("Vp", shape=[128,1],
                            dtype=version1_compat_tensorflow.float32,
                            trainable=True,
                            initializer=version1_compat_tensorflow.glorot_uniform_initializer())
    
    CnnPositionsVector = version1_compat_tensorflow.cast(seq_length-winDowCNNSize,dtype=version1_compat_tensorflow.float32)
 
    Cnnpsv = CnnPositionsVector*version1_compat_tensorflow.nn.sigmoid(version1_compat_tensorflow.matmul(version1_compat_tensorflow.tanh(version1_compat_tensorflow.matmul(CnnDecoderHiddenState,W_par)),V_par))
    Cnnptv = Cnnpsv+Attention_Dimension_Size 
    Cnnptv = version1_compat_tensorflow.reshape(Cnnptv,[batch_size])
    
    i = 0
    CnnGausPosScoreBest = version1_compat_tensorflow.TensorArray(size=seq_length,dtype=version1_compat_tensorflow.float32)
    sigma = version1_compat_tensorflow.constant(Attention_Dimension_Size/2,dtype=version1_compat_tensorflow.float32)
    
    def cond(i,CnnGausPosScoreBest):
        
        return i < seq_length
                      
    def body(i,CnnGausPosScoreBest):
        
        score = version1_compat_tensorflow.exp(-((version1_compat_tensorflow.square(version1_compat_tensorflow.cast(i,version1_compat_tensorflow.float32)-Cnnptv))/(2*version1_compat_tensorflow.square(sigma)))) 
        CnnGausPosScoreBest = CnnGausPosScoreBest.write(i,score)
            
        return i+1,CnnGausPosScoreBest
                      
    i,CnnGausPosScoreBest = version1_compat_tensorflow.while_loop(cond,body,[i,CnnGausPosScoreBest])
    
    CnnGausPosScoreBest = CnnGausPosScoreBest.stack()
    CnnGausPosScoreBest = version1_compat_tensorflow.transpose(CnnGausPosScoreBest,[1,0])
    CnnGausPosScoreBest = version1_compat_tensorflow.reshape(CnnGausPosScoreBest,[batch_size,seq_length])
    
    AttCNNScores = CnnAttentionScore(CnnEncoderStates,CnnDecoderHiddenState)*CnnGausPosScoreBest
    AttCNNScores = version1_compat_tensorflow.nn.softmax(AttCNNScores,axis=-1)
    
    return version1_compat_tensorflow.reshape(AttCNNScores,[batch_size,seq_length,1])

In [ ]:
# defining architecture for lstm decoder with local attention
with version1_compat_tensorflow.variable_scope("decoder",reuse=version1_compat_tensorflow.AUTO_REUSE):
    starting_marker = version1_compat_tensorflow.get_variable("sos", shape=[1,embedding_dimensions],
                                dtype=version1_compat_tensorflow.float32,
                                trainable=True,
                                initializer=version1_compat_tensorflow.glorot_uniform_initializer())

    
    WeightC = version1_compat_tensorflow.get_variable("Wc", shape=[4*hid_lay_cnn,embedding_dimensions],
                            dtype=version1_compat_tensorflow.float32,
                            trainable=True,
                            initializer=version1_compat_tensorflow.glorot_uniform_initializer())
    


starting_marker = version1_compat_tensorflow.tile(starting_marker,[batch_size,1])
inputIteratorMarker = starting_marker
hiddenStateLSTM=FinalEncodedState
CLNcell=version1_compat_tensorflow.zeros([batch_size, 2*hid_lay_cnn], dtype=version1_compat_tensorflow.float32)
CnnLstmDecoderOutput=version1_compat_tensorflow.TensorArray(size=MaxSummaryLen_CNN, dtype=version1_compat_tensorflow.float32)
outputs=version1_compat_tensorflow.TensorArray(size=MaxSummaryLen_CNN, dtype=version1_compat_tensorflow.int32)

CnnAttentionAttCNNScores = align(CnnEncoderStates,hiddenStateLSTM)
encoder_context_vector = version1_compat_tensorflow.reduce_sum(CnnEncoderStates*CnnAttentionAttCNNScores,axis=1)

for iter in range(MaxSummaryLen_CNN):
    
    inputIteratorMarker  = CnnDropout(inputIteratorMarker ,rate=0.3,training=train_tensor_cnn)
    
    inputIteratorMarker  = version1_compat_tensorflow.concat([inputIteratorMarker ,encoder_context_vector],axis=-1)
    
    hiddenStateLSTM,CLNcell = CnnLstm(inputIteratorMarker ,hiddenStateLSTM,CLNcell,embedding_dimensions+2*hid_lay_cnn,2*hid_lay_cnn,scope="decoder")
    
    hiddenStateLSTM = CnnDropout(hiddenStateLSTM,rate=0.3,training=train_tensor_cnn)
    
    CnnAttentionAttCNNScores = align(CnnEncoderStates,hiddenStateLSTM)
    encoder_context_vector = version1_compat_tensorflow.reduce_sum(CnnEncoderStates*CnnAttentionAttCNNScores,axis=1)
    
    concated = version1_compat_tensorflow.concat([hiddenStateLSTM,encoder_context_vector],axis=-1)
    
    linear_out = version1_compat_tensorflow.nn.tanh(version1_compat_tensorflow.matmul(concated,WeightC))
    CnnDecoderOutput = version1_compat_tensorflow.matmul(linear_out,version1_compat_tensorflow.transpose(embed_tensor_cnn,[1,0])) 

    
    CnnLstmDecoderOutput = CnnLstmDecoderOutput.write(iter,CnnDecoderOutput)
    
    
    next_word_vec = version1_compat_tensorflow.cast(version1_compat_tensorflow.argmax(CnnDecoderOutput,1),version1_compat_tensorflow.int32)

    next_word_vec = version1_compat_tensorflow.reshape(next_word_vec, [batch_size])

    outputs = outputs.write(iter,next_word_vec)

    next_word = version1_compat_tensorflow.nn.embedding_lookup(embed_tensor_cnn, next_word_vec)
    inputIteratorMarker  = version1_compat_tensorflow.reshape(next_word, [batch_size, embedding_dimensions])
    
    
CnnLstmDecoderOutput = CnnLstmDecoderOutput.stack()
outputs = outputs.stack()

CnnLstmDecoderOutput = version1_compat_tensorflow.transpose(CnnLstmDecoderOutput,[1,0,2])
outputs = version1_compat_tensorflow.transpose(outputs,[1,0])

In [ ]:
# defining cost function for cross entropy and l2 regularization
filtered_trainables = [var for var in version1_compat_tensorflow.trainable_variables() if
                       not("Bias" in var.name or "bias" in var.name
                           or "noreg" in var.name)]

regularization = version1_compat_tensorflow.reduce_sum([version1_compat_tensorflow.nn.l2_loss(var) for var
                                in filtered_trainables])

with version1_compat_tensorflow .variable_scope("loss"):

    epsilon = version1_compat_tensorflow .constant(1e-9, version1_compat_tensorflow.float32)

    crossEntropy = version1_compat_tensorflow .nn.sparse_softmax_cross_entropy_with_logits(
        labels=sumCNN_Tensor, logits=CnnLstmDecoderOutput)

    MaskPadCnn = version1_compat_tensorflow.sequence_mask(summary_true_tensor_len_cnn,
                                maxlen=MaxSummaryLen_CNN,
                                dtype=version1_compat_tensorflow .float32)

    CnnmaskedCrossEntropy = crossEntropy*MaskPadCnn

    costCnn = version1_compat_tensorflow .reduce_mean(CnnmaskedCrossEntropy) + \
        regL2Param*regularization

    cross_entropy = version1_compat_tensorflow .reduce_mean(CnnmaskedCrossEntropy)

In [ ]:
# compare label and predicted sequence
comparison = version1_compat_tensorflow.cast(version1_compat_tensorflow.equal(outputs, sumCNN_Tensor),
                     version1_compat_tensorflow.float32)
# Ignore effect of pads by masking

MaskPadCnn = version1_compat_tensorflow.sequence_mask(summary_true_tensor_len_cnn,
                            maxlen=MaxSummaryLen_CNN,
                            dtype=version1_compat_tensorflow.bool)

masked_comparison = version1_compat_tensorflow.boolean_mask(comparison,MaskPadCnn)
#Getting accuracy
CnnAccRacy = version1_compat_tensorflow.reduce_mean(masked_comparison)

In [ ]:
variableAllTensor = version1_compat_tensorflow.get_collection(version1_compat_tensorflow.GraphKeys.TRAINABLE_VARIABLES)

CnnOptimizer = version1_compat_tensorflow.train.AdamOptimizer(learning_rate=learnRate_cnn)

gradientVarCnn = CnnOptimizer.compute_gradients(costCnn, variableAllTensor)

cappedGradientVarCnn = [(version1_compat_tensorflow.clip_by_norm(gradientCnn, 5), variableCnn) for gradientCnn, variableCnn in gradientVarCnn] 

OperationTrain = CnnOptimizer.apply_gradients(cappedGradientVarCnn)

In [ ]:
import pickle
import random

with version1_compat_tensorflow.Session() as CnnSession:  
    displayStep = 100
    CnnPatience = 5

    load = input("\nLoad through previous ckpt file ? y/n: ")
    print("")
    saver = version1_compat_tensorflow.train.Saver()

    if load.lower() == 'y':


        saver.restore(CnnSession, 'CNN_summary.ckpt')
        CnnSession.run(version1_compat_tensorflow.global_variables())
        CnnSession.run(version1_compat_tensorflow.tables_initializer())

        with open('CNN_summary.pkl', 'rb') as fp:
            train_data = pickle.load(fp)

        epochsCovered = train_data['epochsCovered']
        bestLoss = train_data['best_loss']
        CnnImp = 0
        

    else:
        bestLoss = 2**30
        CnnImp = 0
        epochsCovered = 0

        initial_state = version1_compat_tensorflow.global_variables_initializer()
        CnnSession.run(initial_state)
        CnnSession.run(version1_compat_tensorflow.tables_initializer())

    CnnEpoch=0
    while (CnnEpoch+epochsCovered)<epochs:
        
        print("\n\nTraining Begins\n\n")
        
        batches_indices = [i for i in range(0, len(train_article_batches))]
        random.shuffle(batches_indices)

        SumTracAcc = 0
        SumTrainloss = 0

        for i in range(0, len(train_article_batches)):
            
            j = int(batches_indices[i])

            costCnn,prediction,\
                acc, _ = CnnSession.run([cross_entropy,
                                   outputs,
                                   CnnAccRacy,
                                   OperationTrain],
                                  feed_dict={article_tensor_cnn: train_article_batches[j],
                                             embed_tensor_cnn: embd,
                                             sumCNN_Tensor: train_summary_batches[j],
                                             summary_true_tensor_len_cnn: train_summary_batches_summary_length[j],
                                             train_tensor_cnn: True})
            
            SumTracAcc += acc
            SumTrainloss += costCnn

            if i % displayStep == 0:
                print("Iter "+str(i)+", Cost Computed = " +
                      "{:.3f}".format(costCnn)+", Accurracy Computed = " +
                      "{:.2f}%".format(acc*100))
            
            if i % 500 == 0:
                
                idx = random.randint(0,len(train_article_batches[j])-1)
                
                
                
                text = " ".join([index_vocabulary_mapping.get(vec,"<UNK>") for vec in train_article_batches[j][idx]])
                predicted_summary = [index_vocabulary_mapping.get(vec,"<UNK>") for vec in prediction[idx]]
                actual_summary = [index_vocabulary_mapping.get(vec,"<UNK>") for vec in train_summary_batches[j][idx]]
                
                
        print("\n\nPerforming checks\n\n")
                
        valTotalLoss=0
        valTotalAccuracy=0
                
        for i in range(0, len(validation_article_batches)):
            
            if i%100==0:
                print("Validating data # {}".format(i))

            costCnn, prediction,\
                acc = CnnSession.run([cross_entropy,
                                outputs,
                                CnnAccRacy],
                                  feed_dict={article_tensor_cnn: validation_article_batches[i],
                                             embed_tensor_cnn: embd,
                                             sumCNN_Tensor: validation_summary_batches[i],
                                             summary_true_tensor_len_cnn: validation_summary_batches_summary_length[i],
                                             train_tensor_cnn: False})
            
            valTotalLoss += costCnn
            valTotalAccuracy += acc
            
        avg_val_loss = valTotalLoss/len(validation_article_batches)
        
        print("\n Epochs covered : {}\n\n".format(CnnEpoch+epochsCovered))
        print("Avg Train Loss: {:.3f}".format(SumTrainloss/len(train_article_batches)))
        print("Avg Train Accuracy: {:.2f}".format(100*SumTracAcc/len(train_article_batches)))
        print("Avg Val Loss: {:.3f}".format(avg_val_loss))
        print("Avg Val Acc: {:.2f}".format(100*valTotalAccuracy/len(validation_article_batches)))
              
        if (avg_val_loss < bestLoss):
            best_loss = avg_val_loss
            save_data={'best_loss':best_loss,'epochsCovered':epochsCovered+CnnEpoch+1}
            CnnImp=0
            with open('CNN_summary.pkl', 'wb') as fp:
                pickle.dump(save_data, fp)
            saver.save(CnnSession, 'CNN_summary.ckpt')
            print("\nModel saved\n")
              
        else:
            CnnImp+=1
              
        if CnnImp > CnnPatience:
              break
              
              
        CnnEpoch+=1




Training Begins


Iter 0, Cost Computed = 1.925, Accurracy Computed = 0.00%
Iter 100, Cost Computed = 0.962, Accurracy Computed = 33.51%
Iter 200, Cost Computed = 0.828, Accurracy Computed = 36.05%
Iter 300, Cost Computed = 0.644, Accurracy Computed = 41.46%
